### This script computes the top 75 results according to bm25

#### It is meant to be used to generate the bm25 results for the hybrid approach

In [4]:
!pip install rank_bm25

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi

In [6]:

PATH_COLLECTION_DATA = '../X_Data/subtask4b_collection_data.pkl'
PATH_QUERY_TEST = '../X_Data/subtask4b_query_tweets_test.tsv'

df_collection = pd.read_pickle(PATH_COLLECTION_DATA)
df_test = pd.read_csv(PATH_QUERY_TEST, sep='\t')

In [7]:
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [8]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:75]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk

In [9]:
df_test['bm25_topk'] = df_test['tweet_text'].apply(lambda x: get_top_cord_uids(x)[:75])

KeyboardInterrupt: 

In [ ]:
df_test.to_csv('bm25_test.tsv', index=None, sep='\t')